In [1]:
install.packages("reshape2", dependencies=TRUE)


Installing package into 'C:/Users/viadu/AppData/Local/R/win-library/4.4'
(as 'lib' is unspecified)



package 'reshape2' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\viadu\AppData\Local\Temp\RtmpKoU306\downloaded_packages


In [2]:
library(reshape2)


Warning message:
"package 'reshape2' was built under R version 4.4.3"


In [3]:
install.packages("ggplot2", dependencies=TRUE)


Installing package into 'C:/Users/viadu/AppData/Local/R/win-library/4.4'
(as 'lib' is unspecified)



package 'ggplot2' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\viadu\AppData\Local\Temp\RtmpKoU306\downloaded_packages


In [4]:
library(ggplot2)


Warning message:
"package 'ggplot2' was built under R version 4.4.3"


In [5]:
# Load necessary library
library(dplyr)

Warning message:
"package 'dplyr' was built under R version 4.4.3"

Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union




# Load Dataset

In [ ]:
df <- read.csv('hospitaldata.csv')

In [ ]:
head(df)

In [ ]:
nrow(df)

# Data Cleaning

In [ ]:
colnames(df)  # Check column names


In [ ]:
# Convert nominal categorical variables to factors
nominal_vars <- c("gender", "race", "medical_specialty", "change", "diabetesMed", "discharge_disposition_id", "admission_source_id")

df[nominal_vars] <- lapply(df[nominal_vars], as.factor)
str(df[nominal_vars])



In [ ]:
levels(df$gender) # Check levels of each nominal variable
levels(df$race)
levels(df$medical_specialty)
levels(df$change)
levels(df$diabetesMed)
levels(df$discharge_disposition_id)
levels(df$admission_source_id)

summary(df[nominal_vars]) # Summarize nominal variables

In [ ]:
# Reference for converting ordinal variables to ordered factors:
# OARC Stats. (n.d.). Factor variables | R Learning Modules.
# URL: https://stats.oarc.ucla.edu/r/modules/factor-variables/

# Convert ordinal categorical variables to ordered factors
df$age <- factor(df$age, levels = c("[0-10)", "[10-20)", "[20-30)", "[30-40)", 
                                     "[40-50)", "[50-60)", "[60-70)", "[70-80)", 
                                     "[80-90)", "[90-100)"), ordered = TRUE)
# Convert to ordered factors
df$max_glu_serum <- factor(df$max_glu_serum, 
                             levels = c(" None", "Norm", ">200", ">300"), 
                             ordered = TRUE)

df$A1Cresult <- factor(df$A1Cresult, 
                         levels = c(" None", "Norm", ">7", ">8"), 
                         ordered = TRUE)

# Reference for processing multiple columns:
# Bhalla, D. (2015). R: Converting Multiple Columns to Factor.
# URL: https://www.listendata.com/2015/05/converting-multiple-numeric-variables.html
# Assign all drugs to one variable drug_vars

drug_vars <- c("metformin", "repaglinide", "nateglinide", "chlorpropamide", "glimepiride", "acetohexamide", "glipizide", "glyburide", "tolbutamide", "pioglitazone", "rosiglitazone", "acarbose", "miglitol", "troglitazone", "tolazamide", "examide", "citoglipton", "insulin", "glyburide_metformin", "glipizide_metformin", "glimepiride_pioglitazone", "metformin_pioglitazone")
df[drug_vars] <- lapply(df[drug_vars], function(x) {
  x <- as.character(x)  # Ensure it's a character vector
  x <- trimws(x)        # Trim leading/trailing spaces
  x[!(x %in% c("No", "Steady", "Down", "Up"))] <- NA  # Replace unexpected values with NA
  factor(x, levels = c("No", "Steady", "Down", "Up"), ordered = TRUE)
})


In [ ]:
unique(unlist(df[drug_vars]))


In [ ]:
unique(df$max_glu_serum)

In [ ]:
unique(df$A1Cresult)

In [ ]:
str(df$max_glu_serum)


In [ ]:
# Check lab test factors
str(df$A1Cresult)


In [ ]:
# Check age factors
str(df$age)

In [ ]:
# Check missing values
colSums(is.na(df))

In [ ]:
df$readmitted_binary <- ifelse(df$readmitted == "No", 0, 1) # Convert to binary for logistic regression
df$readmitted_binary <- as.factor(df$readmitted_binary)  # Convert to factor

In [ ]:
table(df$readmitted_binary)

In [ ]:
df$weight <- NULL # Drop weight column
df$diag_2 <- NULL
df$diag_3 <- NULL
df$encounter_id <- NULL
df$patient_nbr <- NULL
df$drug_vars <- NULL

In [ ]:
# Since most cases have missing HbA1C values
# Merge Categories
# Grouping A1Cresult values into a new variable A1C_Grouped using nested ifelse()
# Reference: https://www.statology.org/ifelse-in-r/
df$A1C_Grouped <- ifelse(df$A1Cresult == " None", "Not Tested",
                         ifelse(df$A1Cresult %in% c(">7", ">8"), "High HbA1c", 
                                ifelse(df$A1Cresult == "Norm", "Normal", NA)))

In [ ]:
class(df)

# Exploratory Data Analysis

In [ ]:
# Summary of numerical columns
summary(df)

# Summary of categorical columns
# summary(df[sapply(data, is.character)])

In [ ]:
sapply(df[sapply(df, is.character)], unique)

In [ ]:
# This function was copied and adapted from Simmons Example provided by Christian Horn in 
# Statistics and Optimisation at NCI, 2025. 
# It generates a histogram with a normal distribution curve overlay.
histogram <- function(x)
{ 
    title <- paste(deparse(substitute(x), 500), collapse="\n") 
    sdx <- sd(x)
    mx <- mean(x)
    hist(x, prob=TRUE, 
         main=paste("Histogram of ",title),
         xlim=c(mx-3*sdx, mx+3*sdx), ylim=c(0, 0.5/sdx))
    curve(dnorm(x, mean=mx, sd=sdx), col='red', lwd=3, add=TRUE)
} # pasted from simmons example

In [ ]:
par(mfrow=c(2,2))
# Histograms
histogram(df$time_in_hospital)
histogram(df$num_medications)
histogram(df$num_lab_procedures)
histogram(df$num_procedures)
# Scatter Plots
plot(readmitted_binary ~ time_in_hospital, df)
plot(readmitted_binary ~ num_medications, df)
plot(readmitted_binary ~ num_lab_procedures, df)
plot(readmitted_binary ~ num_procedures, df)
# Boxplots
boxplot(time_in_hospital ~ readmitted_binary, df)
boxplot(num_medications ~ readmitted_binary, df)

In [ ]:
boxplot(df$time_in_hospital, main="Time in Hospital (Outliers)")
boxplot(df$num_medications, main="Number of Medications (Outliers)")


In [ ]:
# Reference for summarizing categorical variables with frequency tables:
# Nahhas, R. W. (n.d.). Categorical: Bar chart. In An Introduction to R for Research.
# URL: https://bookdown.org/rwnahhas/IntroToR/categorical-bar-chart.html

#categorical variables analysis
table(df$race) #race analysis
barplot(table(df$race), main="Race Distribution")

table(df$gender) #gender analysis
barplot(table(df$gender), main="Gender Distribution")

table(df$age) #age analysis
barplot(table(df$age), main="Age Distribution")

table(df$medical_specialty) #medical specialty analysis
barplot(table(df$medical_specialty), main="Medical Specialty Distribution")

table(df$A1Cresult) #A1C analysis
barplot(table(df$A1Cresult), main="A1Cresult Distribution")


table(df$A1C_Grouped) #Grouped A1C analysis
barplot(table(df$A1C_Grouped), main="A1Cresult Grouped Distribution")



In [ ]:
# Selecting only numeric columns using dplyr's select_if(is.numeric)
# Source: https://www.statology.org/dplyr-select-numeric-columns/

# Calculating correlation matrix using base cor() function
# Source: https://www.rdocumentation.org/packages/stats/topics/cor
cor_matrix <- cor(df %>% select_if(is.numeric), use="complete.obs")
print(cor_matrix)

In [ ]:
names(data)

In [ ]:
# Code adapted from "ggplot2: Quick correlation matrix heatmap - R software and data visualization"
# Source: https://www.sthda.com/english/wiki/ggplot2-quick-correlation-matrix-heatmap-r-software-and-data-visualization

library(ggplot2)
library(reshape2)

melted_cor <- melt(cor_matrix)
ggplot(data = melted_cor, aes(x=Var1, y=Var2, fill=value)) +
  geom_tile() +
  scale_fill_gradient2(low="blue", high="red", mid="white", midpoint=0) +
  theme_minimal() +
  theme(axis.text.x = element_text(angle=90, hjust=1))

In [ ]:
# Code adapted from "R Graphics Cookbook: Making a Proportional Stacked Bar Graph"
# Source: https://r-resources.massey.ac.nz/rgcookbook/RECIPE-BAR-GRAPH-PROPORTIONAL-STACKED-BAR.html
library(ggplot2)
ggplot(df, aes(x=age, fill=as.factor(readmitted_binary))) +
  geom_bar(position="fill") +
  labs(title="Readmission Rate by Age Group", y="Proportion") # Readmission rate by Age

ggplot(df, aes(x=gender, fill=as.factor(readmitted_binary))) +
  geom_bar(position="fill") +
  labs(title="Readmission Rate by Gender", y="Proportion") # Readmission rate by Gender

ggplot(df, aes(x=race, fill=as.factor(readmitted_binary))) +
  geom_bar(position="fill") +
  labs(title="Readmission Rate by Race", y="Proportion") # Readmission rate by Race

ggplot(df, aes(x=admission_source_id, fill=as.factor(readmitted_binary))) +
  geom_bar(position="fill") +
  labs(title="Readmission Rate by Admission Type", y="Proportion") # Readmission rate by Admission Type

ggplot(df, aes(x=medical_specialty, fill=as.factor(readmitted_binary))) +
  geom_bar(position="fill") +
  labs(title="Readmission Rate by Medical Specialty", y="Proportion") #Readmission rate by Medical Specialty

ggplot(df, aes(x=diabetesMed, fill=as.factor(readmitted_binary))) +
  geom_bar(position="fill") +
  labs(title="Readmission Rate by Diabetes Medication", y="Proportion") #Readmission rate by Diabetes Medication

ggplot(df, aes(x=A1Cresult, fill=as.factor(readmitted_binary))) +
  geom_bar(position="fill") +
  labs(title="Readmission Rate by HbA1c Levels", y="Proportion")










In [ ]:
#primary_diagnosis

unique(df$diag_1)



# Logistic Regression

In [ ]:
#Model WITH outliers included
logistic_model <- glm(readmitted_binary ~ time_in_hospital + num_medications, 
                      data = df, family = binomial)
summary(logistic_model)


In [ ]:
# Reference: Patil, P. (2023). Outlier Detection and Removal using the IQR Method.
# URL: https://medium.com/@pp1222001/outlier-detection-and-removal-using-the-iqr-method-6fab2954315d

#Remove outliers
Q1_tih <- quantile(df$time_in_hospital, 0.25)
Q3_tih <- quantile(df$time_in_hospital, 0.75)
IQR_tih <- Q3_tih - Q1_tih
upper_bound_tih <- Q3_tih + 1.5 * IQR_tih

Q1_nm <- quantile(df$num_medications, 0.25)
Q3_nm <- quantile(df$num_medications, 0.75)
IQR_nm <- Q3_nm - Q1_nm
upper_bound_nm <- Q3_nm + 1.5 * IQR_nm

df_cleaned <- df %>% 
  filter(time_in_hospital <= upper_bound_tih, 
         num_medications <= upper_bound_nm)

# Re-run logistic regression without extreme outliers
logistic_model_cleaned <- glm(readmitted_binary ~ time_in_hospital + num_medications, 
                              data = df_cleaned, family = binomial)
summary(logistic_model_cleaned)


In [ ]:
df$A1C_Grouped <- as.factor(df$A1C_Grouped)  # Ensure it's a factor

logistic_model <- glm(readmitted_binary ~ time_in_hospital + num_medications + A1C_Grouped, # Use A1C_grouped for logistic regression
                      data = df, family = binomial)

summary(logistic_model)

In [ ]:
# Reference for ICD-9 code categorization:
# //en.wikipedia.org/wiki/List_of_ICD-9_codes
# Code adopted using reference:
# https://www.rdocumentation.org/packages/icd/versions/4.0.9

# Reference for logistic regression with interaction terms:
# Nahhas, R. W. (2023). Interactions in Logistic Regression.
# URL: https://www.bookdown.org/rwnahhas/RMPH/blr-interaction.html
# Convert diag_1 to numeric

df$diag_1_num <- suppressWarnings(as.numeric(df$diag_1))

df$primary_diagnosis <- ifelse(grepl("^250", df$diag_1), "Diabetes",
                         ifelse((df$diag_1_num >= 390 & df$diag_1_num <= 459) | df$diag_1_num == 785, "Circulatory",
                         ifelse((df$diag_1_num >= 460 & df$diag_1_num <= 519) | df$diag_1_num == 786, "Respiratory",
                         ifelse((df$diag_1_num >= 520 & df$diag_1_num <= 579) | df$diag_1_num == 787, "Digestive",
                         ifelse((df$diag_1_num >= 800 & df$diag_1_num <= 999), "Injury/Trauma", "Other")))))

df$primary_diagnosis <- as.factor(df$primary_diagnosis)


logistic_model_int <- glm(readmitted_binary ~ time_in_hospital + num_medications + A1C_Grouped * primary_diagnosis, # Logistic model with interaction between A1C and primary diagnosis
                      data = df, family = binomial)
summary(logistic_model_int)




In [ ]:
par(mfrow=c(2,2))
plot(logistic_model_int)

In [ ]:
logistic_model_poly <- glm(readmitted_binary ~ time_in_hospital + I(time_in_hospital^2) + # time_in_hospital and num_medicaations might have non-linear effects
                           num_medications + I(num_medications^2) + 
                           A1C_Grouped * primary_diagnosis, 
                           data = df, family = binomial)
summary(logistic_model_poly)
AIC(logistic_model_poly)


In [ ]:
par(mfrow=c(2,2))
plot(logistic_model_poly)

In [ ]:
# Use Log transformation to handle non linearity
df_cleaned$log_time_in_hospital <- log(df_cleaned$time_in_hospital + 1)
df_cleaned$log_num_medications <- log(df_cleaned$num_medications + 1)


In [ ]:
# Reference: Thieme, C. (2020). Identifying Outliers in Linear Regression - Cook's Distance.
# URL: https://rpubs.com/christianthieme/769935

# Compute Cook's Distance
cooksd <- cooks.distance(logistic_model_poly)

# Identify influential points (Cook’s D > 4/N)
influential_points <- as.numeric(names(cooksd[cooksd > (4 / nrow(df))]))

# Remove them from the dataset
df_cleaned <- df[-influential_points, ]


In [ ]:
dim(df_cleaned)

In [ ]:
logistic_model_poly_cleaned <- glm(readmitted_binary ~ log_time_in_hospital + 
                           I(log_time_in_hospital^2) +
                           log_num_medications + I(log_num_medications^2) + 
                           A1C_Grouped * primary_diagnosis, 
                           data = df_cleaned, family = binomial)

summary(logistic_model_poly_cleaned)
AIC(logistic_model_poly_cleaned)


In [ ]:
par(mfrow=c(2,2))
plot(logistic_model_poly_cleaned)

In [ ]:
# Reference: Thieme, C. (2020). Identifying Outliers in Linear Regression - Cook's Distance.
# URL: https://rpubs.com/christianthieme/769935
# Step 1: Fit the initial model with interactions and polynomial terms
logistic_model_initial <- glm(readmitted_binary ~ log_time_in_hospital + 
                               I(log_time_in_hospital^2) +
                               log_num_medications + I(log_num_medications^2) + 
                               A1C_Grouped * primary_diagnosis, 
                               data = df_cleaned, family = binomial)

# Step 2: Compute Cook's Distance
cooksd <- cooks.distance(logistic_model_initial)

# Step 3: Identify high-leverage points (Cook’s D > 4/n)
threshold <- 4 / nrow(df_cleaned)
high_leverage_points <- which(cooksd > threshold)

# Step 4: Remove influential points from the data
df_final <- df_cleaned[-high_leverage_points, ]

# Step 5: Refit the model on the filtered data
logistic_model_poly_final <- glm(readmitted_binary ~ log_time_in_hospital + 
                                  I(log_time_in_hospital^2) +
                                  log_num_medications + I(log_num_medications^2) + 
                                  A1C_Grouped * primary_diagnosis, 
                                  data = df_final, family = binomial)

# Step 6: View model summary and AIC
summary(logistic_model_poly_final)
AIC(logistic_model_poly_final)




In [ ]:
par(mfrow=c(2,2))
plot(logistic_model_poly_final)

In [ ]:
pred_probs <- predict(logistic_model_poly_final, newdata = df_final, type = "response")


In [ ]:
pred_class <- ifelse(pred_probs > 0.5, 1, 0)


In [ ]:
table(Predicted = pred_class, Actual = df_final$readmitted_binary)



In [ ]:
conf_mat <- table(Predicted = pred_class, Actual = df_final$readmitted_binary)
accuracy <- sum(diag(conf_mat)) / sum(conf_mat)
print(accuracy)
